In [2]:
import os
import urllib.request
import re
from urllib.error import HTTPError
assortment = ["Sobtiya", "Politika", "Sotsium", "Proishestviya", "Economy", "Kultura", "Sport", "Omichi"]
adress = "http://omskgazeta.ru/"
def setfolders(location, massive):
    dirlist = []
    for item in massive:
        folder = os.path.join(location, str(item))
        if not os.path.exists(folder):
            os.mkdir(folder)
        dirlist.append(folder)
    return dirlist

def setcatalog():
    book = "papercorpus_plain"
    if not os.path.exists(book):
        os.mkdir(book)
    topicdirlist = setfolders(book, assortment)
    for tfolder in topicdirlist:
        chronology = [x for x in range(2010,2019)]
        yeardirlist = setfolders(tfolder, chronology)
        for yfolder in yeardirlist:
            calendar = [x for x in range(1,13)]
            monthdirlist = setfolders(yfolder, calendar)
            m31 = [0, 2, 4, 6, 7, 9, 11]
            m30 = [3, 5, 8, 10]
            for i, mfolder in enumerate(monthdirlist):
                if i in m31:
                    diary = [x for x in range(1, 32)]
                elif i in m30:
                    diary = [x for x in range(1, 31)]
                else:
                    diary = [x for x in range(1, 30)]
                daydirlist = setfolders(mfolder, diary)
    return book

def distributetheme(pageurl):
    blocks = pageurl.split("/")
    namedir = blocks[3]
    return namedir

def givename(pageurl):
    blocks = pageurl.split("/")
    namefile = blocks[-2]
    return namefile
    
def downloadpage(pageurl):
    page = urllib.request.urlopen(pageurl)
    text = page.read().decode('utf-8')
    return text

def preparefile():
    filename = os.path.join(setcatalog(), "metadata.tsv")
    with open(filename, "w", encoding="utf-8") as f:
        f.write("path\tauthor\theader\tcreated\tsphere\ttopic\tstyle\taudience_age\taudience_level\taudience_size\tsource\tpublication\tpubl_year\tmedium\tcountry\tregion\tlanguage")
    return filename

def writepage(pageurl, findir):
    rudict = {"января": 1, "февраля": 2, "марта": 3, "апреля": 4, "мая": 5, "июня": 6, "июля": 7, "августа": 8, "сентября": 9, "октября": 10, "ноября": 11, "декабря": 12}
    pagetext = downloadpage(pageurl)
    timeline = re.search('[0-2][0-9]:[0-5][0-9], ([1-3]?[0-9]) (.+?) (201[0-8])', pagetext)
    year = timeline.group(3)
    month = timeline.group(2)
    day = timeline.group(1)
    yenum = str(year)
    monnum = str(rudict[month])
    danum = str(day)
    fileplace = os.path.join(setcatalog(), findir, yenum, monnum, danum)
    fileownname = givename(pageurl)
    filename = os.path.join(fileplace, fileownname)
    with open(filename, "w", encoding="utf-8") as f:
        f.write("@au None")
        f.write("@ti " + fileownname)
        f.write("@da " + danum + "." + monnum + "." + yenum)
        f.write("@topic " + findir)
        f.write("@url " + pageurl)
        f.write(pagetext)
    metafile = preparefile()
    with open(metafile, "a", encoding="utf-8"):
        f.write(filename + "\t" + "unknown" + "\t" + fileownname + danum +"." + monnum + "." + yenum + "\t" + "публицистика\t" + findir + "\t" + "нейтральный\t" + "н-возраст\t" + "н-уровень\t" + "городская\t" + pageurl + "\t" + "Газета-Омск\t" + yenum + "\t" + "газета\t" + "Россия\t" + "Омск\t" + "ru")
    
def loadcatalog(topicurl):
    pageslist = []
    thematicnews = []
    number = 1
    blocks = topicurl.split("/")
    theme = blocks[3]
    tema = theme.lower()
    while number:
        pageurl = topicurl + "?page=" + str(number) + "/"
        pagetext = downloadpage(pageurl)
        if not re.search("В категории нет новостей", pagetext):
            number = number + 1
            pageslist.append(pagetext)
        else:
            break
    for pagetext in pageslist:
        refs = re.findall('href=".+?/.+?/"', pagetext)
        neededrefs = []
        for ref in refs:
            subrefs = ref.split('"')
            pathtext = subrefs[1]
            if re.search(tema, pathtext):
                neededrefs.append(pathtext)
    for ppath in neededrefs:
            adresstext = adress + ppath
            print(adresstext)
            note = downloadpage(adresstext)
            thematicnews.append(note)
    return thematicnews

def fillcatalog():
    for topic in assortment:
        topicurl = adress + topic + "/"
        subcorpus = loadcatalog(topicurl)
        for pagetext in subcorpus:
            section = re.search('"v:Breadcrumb"><a href="(.+?)/" >', pagetext)
            findir = section.group(1)
            location = re.search('"v:Breadcrumb"><a href="(.+?/.+?)/" class="last">', pagetext)
            url = location.group(1)
            writepage(url, findir)
            

def main():
    setcatalog()
    corpus = loadcorpus(adress)
    fillcatalog()
main()
    

http://omskgazeta.ru/sobtiya/roman_frantsuza_stal_biletom_v_parij/
http://omskgazeta.ru/sobtiya/roman_frantsuza_stal_biletom_v_parij/
http://omskgazeta.ru/sobtiya/latshi_otmetili_ligo/
http://omskgazeta.ru/sobtiya/latshi_otmetili_ligo/
http://omskgazeta.ru/sobtiya/razne_kultur__odna_sudba/
http://omskgazeta.ru/sobtiya/razne_kultur__odna_sudba/
http://omskgazeta.ru/sobtiya/v_eti_vxodne_omichi_uvidyat_serebriste_oblaka/
http://omskgazeta.ru/sobtiya/v_eti_vxodne_omichi_uvidyat_serebriste_oblaka/
http://omskgazeta.ru/sobtiya/prazdniku_rad_i_mal_i_star/
http://omskgazeta.ru/sobtiya/prazdniku_rad_i_mal_i_star/
http://omskgazeta.ru/sobtiya/v_omske_poyavitsya_chuvstvitelny_lokator/
http://omskgazeta.ru/sobtiya/v_omske_poyavitsya_chuvstvitelny_lokator/
http://omskgazeta.ru/sobtiya/blagotvoritelny_kontsert_pamyati_arkadiya_kutilova/
http://omskgazeta.ru/sobtiya/blagotvoritelny_kontsert_pamyati_arkadiya_kutilova/
http://omskgazeta.ru/sobtiya/ilya_chert_jit_v_rossii_jutko_no_interesno/
http://omsk

ValueError: unknown url type: '" >Газета - Омск</a></span> <span>&rarr;</span> <span property="v:title" rel="v:url" typeof="v:Breadcrumb"><a href="sobtiya/" >События</a></span> <span>&rarr;</span> <span property="v:title" rel="v:url" typeof="v:Breadcrumb"><a href="sobtiya/roman_frantsuza_stal_biletom_v_parij'